In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.15.0


In [2]:
fashion_mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
num_classes = len(np.unique(train_labels))

In [4]:
def data_processing(train_images, test_images):
  train_images = tf.cast(train_images, tf.float32)
  test_images = tf.cast(test_images, tf.float32)
  train_images /= 255
  test_images /=255
  return train_images, test_images

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

def augmentation_data(train_images, train_labels, test_images, test_labels, batch_size=32):
    # Chuyển tensors sang mảng NumPy
    train_images = train_images.numpy()
    test_images = test_images.numpy()

    # Reshape lại dữ liệu để phù hợp với yêu cầu của ImageDataGenerator
    train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
    test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

    # Chuyển đổi dữ liệu thành định dạng float32 và chuẩn hóa
    train_images = train_images.astype('float32') / 255
    test_images = test_images.astype('float32') / 255

    # Khởi tạo ImageDataGenerator với các phép biến đổi
    datagen = ImageDataGenerator(
        rotation_range=20,  # Góc xoay ảnh (độ)
        width_shift_range=0.2,  # Dịch chuyển chiều rộng
        height_shift_range=0.2,  # Dịch chuyển chiều cao
        zoom_range=0.2,  # Phóng to và co lại ảnh
        horizontal_flip=True  # Lật ảnh theo chiều ngang
    )

    # Fit generator với dữ liệu huấn luyện
    datagen.fit(train_images)

    # Tiến hành augmentation dữ liệu và tạo ra các batch mới
    augmented_train_data = datagen.flow(train_images, train_labels, batch_size=batch_size)

    # Tiến hành augmentation dữ liệu cho tập kiểm tra (không cần phải fit)
    augmented_test_data = datagen.flow(test_images, test_labels, batch_size=batch_size)

    return augmented_train_data, augmented_test_data


In [6]:
def augmentation_preprocessing_data(train_images, train_labels, test_images, test_labels):
   train_images,test_images = data_processing(train_images, test_images)
   augmented_train_data, augmented_test_data = augmentation_data(train_images, train_labels, test_images, test_labels, batch_size=32)
   return augmented_train_data, augmented_test_data

# Chuyển tensors sang tensors NumPy
train_images = tf.convert_to_tensor(train_images)
test_images = tf.convert_to_tensor(test_images)
augmented_train_data, augmented_test_data = augmentation_preprocessing_data(train_images, train_labels, test_images, test_labels)



In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD,RMSprop

In [8]:
augmented_train_data[0][0].shape

(32, 28, 28, 1)

In [9]:
model = Sequential([
    Flatten(input_shape = (28,28,1)),

    Dense(256,activation = 'relu'),
    Dense(128,activation = 'relu'),
    Dense(64,activation = 'relu'),
     Dense(32,activation = 'relu'),
    Dense(num_classes,activation = 'softmax'),
])

In [10]:
els = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [11]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)


In [12]:
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

In [13]:
model.fit(augmented_train_data, validation_data=augmented_test_data, epochs = 25, steps_per_epoch=len(train_images) // batch_size,  validation_steps=len(test_images) // batch_size,callbacks=[tensorboard_callback])

Epoch 1/25
1875/1875 [==============================] - 37s 18ms/step - loss: 2.2313 - acc: 0.1553 - val_loss: 2.1017 - val_acc: 0.2115
Epoch 2/25
1875/1875 [==============================] - 31s 16ms/step - loss: 2.0416 - acc: 0.2383 - val_loss: 1.9334 - val_acc: 0.2708
Epoch 3/25
1875/1875 [==============================] - 30s 16ms/step - loss: 1.8010 - acc: 0.3213 - val_loss: 1.6643 - val_acc: 0.3766
Epoch 4/25
1875/1875 [==============================] - 32s 17ms/step - loss: 1.4656 - acc: 0.4620 - val_loss: 1.2371 - val_acc: 0.5556
Epoch 5/25
1875/1875 [==============================] - 30s 16ms/step - loss: 1.2051 - acc: 0.5681 - val_loss: 1.0951 - val_acc: 0.6208
Epoch 6/25
1875/1875 [==============================] - 31s 17ms/step - loss: 1.0537 - acc: 0.6275 - val_loss: 1.0493 - val_acc: 0.6239
Epoch 7/25
1875/1875 [==============================] - 31s 17ms/step - loss: 0.9328 - acc: 0.6741 - val_loss: 0.8411 - val_acc: 0.7124
Epoch 8/25
1875/1875 [==========================

In [ ]:
list(dataset.take(1))[0]